### Load libraries and functions

In [ ]:

from scipy.spatial.distance import jensenshannon
import numpy as np
import sys

sys.path.insert(0, '../src/')

from load_modify_sample_utils import load_model, get_objective_functions, get_reaction_bounds, modify_model
from load_modify_sample_utils import sample_optgp

from loopless_utils import loops_enumeration_from_fva

from correlations_utils import compute_copula, copula_tail_dependence
from correlations_utils import plot_copula


### Load and inspect model (for more info see `load_modify_samply.ipynb`)

In [2]:

ec_cobra_model, ec_cobra_reactions, ec_cobra_reaction_ids,  = load_model("../ext_data/models/e_coli_core.xml")

objective_functions = get_objective_functions(ec_cobra_model)
print(objective_functions)

default_reaction_bounds = get_reaction_bounds(ec_cobra_model)
print(default_reaction_bounds)


Set parameter Username
Set parameter LicenseID to value 2642044
Academic license - for non-commercial use only - expires 2026-03-25
['BIOMASS_Ecoli_core_w_GAM']
{'PFK': (0.0, 1000.0), 'PFL': (0.0, 1000.0), 'PGI': (-1000.0, 1000.0), 'PGK': (-1000.0, 1000.0), 'PGL': (0.0, 1000.0), 'ACALD': (-1000.0, 1000.0), 'AKGt2r': (-1000.0, 1000.0), 'PGM': (-1000.0, 1000.0), 'PIt2r': (-1000.0, 1000.0), 'ALCD2x': (-1000.0, 1000.0), 'ACALDt': (-1000.0, 1000.0), 'ACKr': (-1000.0, 1000.0), 'PPC': (0.0, 1000.0), 'ACONTa': (-1000.0, 1000.0), 'ACONTb': (-1000.0, 1000.0), 'ATPM': (8.39, 1000.0), 'PPCK': (0.0, 1000.0), 'ACt2r': (-1000.0, 1000.0), 'PPS': (0.0, 1000.0), 'ADK1': (-1000.0, 1000.0), 'AKGDH': (0.0, 1000.0), 'ATPS4r': (-1000.0, 1000.0), 'PTAr': (-1000.0, 1000.0), 'PYK': (0.0, 1000.0), 'BIOMASS_Ecoli_core_w_GAM': (0.0, 1000.0), 'PYRt2': (-1000.0, 1000.0), 'CO2t': (-1000.0, 1000.0), 'RPE': (-1000.0, 1000.0), 'CS': (0.0, 1000.0), 'RPI': (-1000.0, 1000.0), 'SUCCt2_2': (0.0, 1000.0), 'CYTBD': (0.0, 1000.

### Modify the model to create two different conditions (for more info see `load_modify_samply.ipynb`)

In [3]:

# Set optimal percentage to 100
ec_cobra_model_condition_100 = modify_model(
    cobra_model         = ec_cobra_model,
    objective_function  = "BIOMASS_Ecoli_core_w_GAM",
    optimal_percentage  = 100,
    objective_direction = "max"
)

updated_objective_functions = get_objective_functions(ec_cobra_model_condition_100)
print(updated_objective_functions)

updated_reaction_bounds = get_reaction_bounds(ec_cobra_model_condition_100)
print(updated_reaction_bounds.get("BIOMASS_Ecoli_core_w_GAM"))

# -----------

# Set optimal percentage to 0
ec_cobra_model_condition_0 = modify_model(
    cobra_model         = ec_cobra_model,
    objective_function  = "BIOMASS_Ecoli_core_w_GAM",
    optimal_percentage  = 0,
    objective_direction = "max"
)

updated_objective_functions = get_objective_functions(ec_cobra_model_condition_0)
print(updated_objective_functions)

updated_reaction_bounds = get_reaction_bounds(ec_cobra_model_condition_0)
print(updated_reaction_bounds.get("BIOMASS_Ecoli_core_w_GAM"))



Read LP format model from file /tmp/tmph8_9hsmc.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros


['BIOMASS_Ecoli_core_w_GAM']
(0.872922, 1000)
Read LP format model from file /tmp/tmpmc2t726a.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
['BIOMASS_Ecoli_core_w_GAM']
(0.0, 1000)


### Identify loopy reactions in model (for more info see `loopless.ipynb`)

In [ ]:

loopy_reactions_fva_100 = loops_enumeration_from_fva(ec_cobra_model_condition_100, fraction_of_optimum=0.999)
print(loopy_reactions_fva_100)

loopy_reactions_100 = [item[0] for item in loopy_reactions_fva_100]
print(loopy_reactions_100)


loopy_reactions_fva_0 = loops_enumeration_from_fva(ec_cobra_model_condition_0, fraction_of_optimum=0)
print(loopy_reactions_fva_0)

loopy_reactions_0 = [item[0] for item in loopy_reactions_fva_0]
print(loopy_reactions_0)


### Remove loopy reactions from the 2 models created above to reduce the thermodynamically infeasible solutions from sampling

In [4]:

ec_cobra_model_condition_100.reactions.get_by_id("FRD7").bounds = (0, 0)
ec_cobra_model_condition_0.reactions.get_by_id("FRD7").bounds = (0, 0)


### Perform sampling on the modified models with the loopy reaction "FRD7" removed. (for more info see `load_modify_samply.ipynb`)

In [7]:

samples_optgp_condition_100 = sample_optgp(ec_cobra_model_condition_100, 
                                           n_samples = 3000, 
                                           thinning=100, 
                                           reaction_in_rows = True)


samples_optgp_condition_0 = sample_optgp(ec_cobra_model_condition_0, 
                                         n_samples = 3000, 
                                         thinning=100, 
                                         reaction_in_rows = True)


Read LP format model from file /tmp/tmpf6ug3b8w.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
Read LP format model from file /tmp/tmpp515rra3.lp
Reading time = 0.01 seconds
: 72 rows, 190 columns, 720 nonzeros


### Compute and visualize a copula from a reactions pair

The `plot_copula` function vizualises the copula from the selected pair of reactions. More information on copulas can be found [here](https://doi.org/10.20382/jocg.v14i1a8)

Observation of the plot, shows a peak in the edge where `PGK` takes its lower and `PFK` its higher values. However, we get another signal at the area where `PGK` takes its higher and `PFK` its lower values.

In [28]:

n1 = ec_cobra_reaction_ids.index("PGK")
n2 = ec_cobra_reaction_ids.index("PFK")

flux1 = samples_optgp_condition_0[n1]
flux2 = samples_optgp_condition_0[n2]

data_flux1=[flux1, ec_cobra_reaction_ids[n1]]
data_flux2=[flux2, ec_cobra_reaction_ids[n2]]

cells = 5
plot_copula(data_flux1, data_flux2, n = cells)


### Compute the copula matrix

The `compute_copula` function computes and returns the copula matrix, without visualizing it. Thus we can work on this matrix and extract information

This matrix has dimensions of rows and columns equal to the `cells` parameter we defined above. To work with the copula matrix and extract dependence information we need to define some concepts on its structure. Observation of the printed numpy array below will help with understanding these concepts.

- A 5*5 copula for our algorithm has 4 separate edges/areas:
    - Top left, for example cells close to: [0,0], [0,1], [1,0] matrix position
    - Bottom right, for example cells close to: [4,4], [4,3], [3,4] matrix position
    - Top right, for example cells close to: [4,0], [3,0], [4,1] matrix position
    - Bottom left, for example cells close to: [0,4], [0,3], [1,4] matrix position

- A 5*5 copula for our algorithm has 2 diagonals: 
    - 1st diagonal: includes top left and bottom right areas
    - 2nd diaognal: includes top right and bottom left areas

In [ ]:
cells = 5
copula = compute_copula(flux1, flux2, n = cells)
print(copula)


[[0.001      0.03766667 0.05       0.05433333 0.057     ]
 [0.015      0.04733333 0.04333333 0.048      0.04633333]
 [0.03       0.05       0.046      0.036      0.038     ]
 [0.05766667 0.03833333 0.035      0.034      0.035     ]
 [0.09633333 0.02666667 0.02566667 0.02766667 0.02366667]]


### Compute non-linear dependencies using copulas

Below a test function shows how the corresponding `copula_tail_dependence` works in `correlations_utils.py`.

This function parses the 2 copula's diagonals and picks values from the 4 copula's edges. One diagonal (the 1st) corresponds to the mass probability of 2 reactions working together at the same time (when one is active the other is active too) and the other diagonal (the 2nd) corresponds to the mass propability of 2 reactions not working together at the same time (when one is active the other is inactive).

The `cop_coeff` parameters define the width of the diagonals for the algortihm to parse (leading to more wide or narrow diagonals).

Then, by dividing the sum of propability mass of the 1st to the 2nd diagonal the indicator value is calculated. This value informs us on which diagonal has a higher mass concentration. If the indicator value is above a significance cutoff, this gives a sign to the dependence (positive for indicator value above given threshold or negative for indicator value below the reciprocal of the given threshold).

Division of the corresponding edges (e.g. for the 1st diagonal: top left with the bottom right edge) shows whether higher mass concentration appears in one or both edges. This helps with classification of the copula dependence.

We have different classification types based on:
- sign: `positive` or `negative`, as explained above
- tail(s): show which tail or tails have significant higher mass concentration based on the sign:
    - `positive_upper_tail` for higher mass in the top left area
    - `positive_lower_tail` for higher mass in the bottom right area
    - `positive_upper_lower_tail` for higher mass in both bottom right and top left areas
    
    - `negative_upper_tail` for higher mass in the top right area
    - `negative_lower_tail` for higher mass in the bottom left area
    - `negative_upper_lower_tail` for higher mass in both top right and bottom left areas


Here, we see that we get a `negative_upper_lower_tail` classification for our copula and an indicator value lower than 1. This supports information from the copula visualization, as the 2nd diagonal had clearly higher mass concentration than the 1st. 

For the classification, we could say that the bottom left edge had a significantly higher mass concentration than the top right one and thus the classification could also be: `negative_lower_tail`. However, the peak of the propability mass is concentrated in a single cell only in the bottom left edge (see printed copula matrix above) and thus the algorithm finds significant dependence in both tails.

In [47]:

cop_coeff = 0.2
cop_coeff_1 = cop_coeff
cop_coeff_2 = 1 - cop_coeff
cop_coeff_3 = 1 + cop_coeff

dependence, indicator = copula_tail_dependence(copula = copula, 
                                               cop_coeff_1 = cop_coeff_1, 
                                               cop_coeff_2 = cop_coeff_2, 
                                               cop_coeff_3 = cop_coeff_3, 
                                               indicator_cutoff = 1.2)

print("Dependence:", dependence, "\nIndicator", indicator)


Dependence: negative_upper_lower_tail 
Indicator 0.5211930937491529


### Provide a quantitative metric for dependence strength

From the chunks above, we saw how to identify and provide a sign to a non-linear dependence from the copula matrix. However, we still don't have a value that will inform us on how powerful this dependence is. The `jensenshannon` distance (scaled from 0 to 1) is used to provide this metric, by calculating the distance between our copula and a hypothetical copula of identicla dimensions and uniform distribution (equal values across its cells). 

To show how this distance metric works we will calculate it with the copula we worked above. However, in later chunks more tests are performed

In [49]:

cells = 5

# our copula melted in a 1D array
copula_flat = copula.flatten()

# a reference-uniform copula with same dimensions
reference_copula = np.full( (cells, cells), (1 / (cells*cells)) )
reference_copula_1_flat = reference_copula.flatten()


np.set_printoptions(threshold=np.inf)
print(reference_copula)
print("\n")


dist = jensenshannon(copula_flat, reference_copula_1_flat)
print("Distance between our copula and a hypothetical-uniform one:", dist)


[[0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]
 [0.04 0.04 0.04 0.04 0.04]]


Distance between our copula and a hypothetical-uniform one: 0.1669516998140334


### More test to show how the `jensenshannon` works

We first define the `a` copula, a uniform one with 100 cells. 

Then we create 2 other copulas filled with 0, but modified to have all mass concentrated in the top left edge:
- `b` copula has an extremely sharp peak (all mass is in a single cell)
- `c` copula has a less exteme peak, the mass is concentrated across a wider area

The `b` copula is expected to have a higher distance value, compared to `c`, as it's structure is farther from the structure of the uniform copula `a`. 

In [50]:

cells = 100

a = np.full( (cells, cells), (1 / (cells*cells)) )
a_flat = a.flatten()


b = np.zeros((cells,cells))
b[0,0] = 1
b_flat = b.flatten()


c = np.zeros((cells,cells))
c[:10, :10] = 0.01
c_flat = c.flatten()


In [ ]:

dist_a_b = jensenshannon(a_flat, b_flat)
print(dist_a_b)


dist_a_c = jensenshannon(a_flat, c_flat)
print(dist_a_c)


0.8322479564657576
0.8155344336992489
